In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
sys.path.append("../utils")
from model_utils import load_model, load_baseline, generate, translate_all
from data_utils import get_bbt_data

src_texts, tar_texts, prefixed = get_bbt_data(prefix=">>nob<<")

baseline_model, baseline_tokenizer = load_baseline()
baseline_translations = translate_all(src_texts, baseline_model, baseline_tokenizer)

100%|██████████| 84/84 [00:10<00:00,  8.20it/s]


In [5]:
from evaluator import Evaluator

src_texts, tar_texts, prefixed = get_bbt_data(prefix=">>nob<<")

In [ ]:
import os
import csv
from tqdm.notebook import tqdm

evaluator = Evaluator()
lang = "no"

for compression_ratio in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    print(f"Compression ratio: {compression_ratio}")
    filename = f"BBT/comp_{compression_ratio}_results.csv"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    print(filename)
    with open(filename, "w", newline="") as f:
        model, tokenizer = load_model(compression_ratio)
        translated = translate_all(src_texts, model, tokenizer)

        csvdata = evaluator.get_csv_data(
            predictions=translated,
            references=tar_texts,
            source=src_texts,
            baselines=baseline_translations,
            lang=lang,
        )

        writer = csv.writer(f)
        for data in tqdm(csvdata):
            writer.writerow(data)

In [9]:
import re
import os
import pandas as pd

eval_path = "BBT"
search = lambda x: re.search("comp_(.*)_results.csv", x).group(1)
decimals = 2

translations = {}
metrics = {}
source = []
target = []
baseline = []
baseline_metrics = []
for file in sorted(os.listdir(eval_path), key=search):
    if file.endswith(".csv"):
        compression = search(file)
        df = pd.read_csv(os.path.join(eval_path, file))
        translations[compression] = df["Translation"].tolist()
        metrics[compression] = df["Metrics"].tolist()
        if len(baseline) == 0:
            source = df["Source Text"].tolist()
            target = df["Original Target"].tolist()
            baseline = df["Baseline"].tolist()
            baseline_metrics = df["Baseline Metrics"].tolist()

with open("BBT/metric_summary.txt", "w") as f:
    for i in range(len(baseline)):
        s = f"Source: {source[i]}"
        s += f"\nTarget: {target[i]}"
        s += "\n" + "- - "*25
        for comp_rate in translations.keys():
            s += f"\n- {comp_rate}: {translations[comp_rate][i]}"
            _metrics = eval(metrics[comp_rate][i])
            # _bleu = _metrics["bleu"]
            # _r2 = _metrics["r2"]
            # _meteor = _metrics["meteor"]
            # _bert = _metrics["bertscore"]
            # metric_str = f"BLEU: {round(_bleu, decimals)}, R2: {round(_r2, decimals)}, METEOR: {round(_meteor, decimals)}, BERTSCORE: {round(_bert, decimals)}"
            # s += f"\nMetrics {comp_rate}: {metric_str}"
            s += f"\nMetrics:\n{_metrics}"
        s += "\n" + "- - "*25
        s += f"\nBaseline: {baseline[i]}"
        _baseline_metrics = eval(baseline_metrics[i])
        # _baseline_metric_str = f"BLEU: {round(_baseline_metrics['bleu'], decimals)}, R2: {round(_baseline_metrics['r2'], decimals)}, METEOR: {round(_baseline_metrics['meteor'], decimals)}, BERTSCORE: {round(_baseline_metrics['bertscore'], decimals)}"
        # s += f"\nBaseline Metrics: {_baseline_metric_str}"
        s += f"\nBaseline metrics:\n{_baseline_metrics}"
        s += "\n" + "=="*50
        s += "\n"
        # print(s)
        f.write(s)
        f.write("\n")